In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pheochromocytoma_and_Paraganglioma"
cohort = "GSE33371"

# Input paths
in_trait_dir = "../../input/GEO/Pheochromocytoma_and_Paraganglioma"
in_cohort_dir = "../../input/GEO/Pheochromocytoma_and_Paraganglioma/GSE33371"

# Output paths
out_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/GSE33371.csv"
out_gene_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/gene_data/GSE33371.csv"
out_clinical_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/clinical_data/GSE33371.csv"
json_path = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Beta-catenin status effects in human adrenocortical carcinomas (33), adenomas (22), and normal adrenal cortex (10)"
!Series_summary	"We scored adrenocortical carcinomas and adenomas for abnormal beta-catenin staining, and sequenced the beta-catenin gene in some samples. We compared adrenocortincal carcinomas with and without abnormal beta-catenin staining and found many significant expression differences and significant results from enrichment testing. A similar comparison in the adenomas gave relatively few differences, and they did not correlate to differences found for the carcinomas.  Abnormal beta-catenin staining was associated with mitotic rate and poorer patient survival in the carcinomas.  In a second independent data set (given in a supplement) we again found beta-catenin associated with poor survival.  The array data given is the same as GEO series GSE10927, with additional characteristics about beta-catenin, and new patient followup da

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine gene expression data availability
# Check if this dataset contains gene expression data (not miRNA or methylation data)
# From the background information, we can see this dataset contains "mRNA assays performed using Affymetrix HG_U133_plus_2 arrays"
# This confirms it has gene expression data
is_gene_available = True

# 2. Determine variable availability and create conversion functions

# 2.1 For trait (Pheochromocytoma_and_Paraganglioma)
# Looking at the clinical characteristics in row 3, we can see entries related to pheochromocytoma
# These include: "Adrenalectomy for pheochromocytoma", "Right pheochromocytoma", etc.
trait_row = 3

def convert_trait(value):
    if value is None:
        return None
    # Extract the value after the colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check if the value mentions pheochromocytoma
    if "pheochromocytoma" in value.lower():
        return 1  # Positive for trait
    else:
        return 0  # Negative for trait

# 2.2 For age
# Age data is available in row 0
age_row = 0

def convert_age(value):
    if value is None:
        return None
    # Extract the value after the colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to numeric value
    if value.lower() == "<10":
        # For age less than 10, use 5 as an approximation
        return 5
    try:
        return float(value)
    except:
        return None

# 2.3 For gender
# Gender data is available in row 1 as "Sex: F" or "Sex: M"
gender_row = 1

def convert_gender(value):
    if value is None:
        return None
    # Extract the value after the colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary (Female=0, Male=1)
    if value.upper() == "F":
        return 0
    elif value.upper() == "M":
        return 1
    else:
        return None

# 3. Save metadata about data availability
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # Use the library function to extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(clinical_df)
    print("Preview of extracted clinical data:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to a CSV file
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical data:
{'GSM825367': [0.0, 71.0, 0.0], 'GSM825368': [0.0, 58.0, 0.0], 'GSM825369': [0.0, 71.0, 1.0], 'GSM825370': [0.0, 44.0, 1.0], 'GSM825371': [0.0, 32.0, 0.0], 'GSM825372': [1.0, 28.0, 1.0], 'GSM825373': [0.0, 55.0, 1.0], 'GSM825374': [0.0, 78.0, 1.0], 'GSM825375': [0.0, 41.0, 1.0], 'GSM825376': [1.0, 58.0, 1.0], 'GSM825377': [0.0, 45.0, 0.0], 'GSM825378': [0.0, 28.0, 0.0], 'GSM825379': [0.0, 57.0, 0.0], 'GSM825380': [0.0, 25.0, 0.0], 'GSM825381': [0.0, 25.0, 0.0], 'GSM825382': [0.0, 56.0, 0.0], 'GSM825383': [0.0, 33.0, 0.0], 'GSM825384': [0.0, 44.0, 1.0], 'GSM825385': [0.0, 71.0, 0.0], 'GSM825386': [0.0, 56.0, 0.0], 'GSM825387': [0.0, 51.0, 0.0], 'GSM825388': [0.0, 32.0, 0.0], 'GSM825389': [0.0, 38.0, 1.0], 'GSM825390': [0.0, 49.0, 0.0], 'GSM825391': [0.0, 62.0, 0.0], 'GSM825392': [0.0, 59.0, 1.0], 'GSM825393': [0.0, 52.0, 1.0], 'GSM825394': [1.0, 54.0, 0.0], 'GSM825395': [0.0, 48.0, 1.0], 'GSM825396': [0.0, 55.0, 1.0], 'GSM825397': [0.0, 64.0, 0.0], 'G

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers in the provided list
# These identifiers appear to be Affymetrix probe set IDs (e.g., '1007_s_at', '1053_at')
# which are not human gene symbols but probe identifiers from microarray platforms
# They need to be mapped to official gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe the gene identifiers in expression data and annotation data
# In the gene expression data, the identifiers are like '1007_s_at', '1053_at', etc.
# In the gene annotation data, the 'ID' column contains the same identifiers
# The gene symbols are in the 'Gene Symbol' column

# 2. Get the gene mapping dataframe by extracting the relevant columns
prob_col = 'ID'  # Column name for identifiers
gene_col = 'Gene Symbol'  # Column name for gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression data
# This function will distribute probe values among genes and then sum by gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Display the first few rows of the resulting gene expression dataframe
print("Preview of gene expression data after mapping:")
print(gene_data.index[:20])  # Show first 20 gene symbols after mapping


Preview of gene expression data after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06', 'AAAS', 'AACS', 'AACSP1', 'AADAC', 'AADACL2',
       'AADACP1', 'AADAT', 'AAED1', 'AAGAB', 'AAK1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data from the previously saved file
clinical_df = pd.read_csv(out_clinical_data_file)

# First check the structure of the loaded clinical data
print(f"Loaded clinical data shape: {clinical_df.shape}")

# Create a proper DataFrame for linking - we need to transpose the clinical data
# and set column names for the clinical features
clinical_features = [trait, 'Age', 'Gender']
samples = clinical_df.columns

# Convert sample-wise columns to sample-wise rows with proper clinical feature columns
clinical_data_transpose = pd.DataFrame(clinical_df.values.T, index=samples, columns=clinical_features)
print(f"Transposed clinical data shape: {clinical_data_transpose.shape}")

# Link the clinical and genetic data by sample IDs
linked_data = clinical_data_transpose.join(normalized_gene_data.T)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data for adrenocortical samples with pheochromocytoma classification."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data saved to ../../output/preprocess/Pheochromocytoma_and_Paraganglioma/gene_data/GSE33371.csv
Loaded clinical data shape: (3, 65)
Transposed clinical data shape: (65, 3)
Linked data shape: (65, 19848)


Shape after handling missing values: (65, 19848)
For the feature 'Pheochromocytoma_and_Paraganglioma', the least common label is '1.0' with 3 occurrences. This represents 4.62% of the dataset.
The distribution of the feature 'Pheochromocytoma_and_Paraganglioma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 35.0
  50% (Median): 48.0
  75%: 56.0
Min: 5.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 24 occurrences. This represents 36.92% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data quality check failed. The dataset is not suitable for association studies.
